In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier



In [3]:
# List files in the directory
files = os.listdir('C:\\Users\\Administrator\\Downloads\\Match prediction\\Dataset')

# Print the file names
print(files)


['Match 2022.xlsx', 'Match 2023.xlsx', 'Match 2024.xlsx']


In [4]:
# Read data
df = pd.concat([pd.read_excel(f"C:\\Users\\Administrator\\Downloads\\Match prediction\\Dataset\{file}", index_col =0) for file in files])


In [5]:
df.head()


,Time,League,Country,Home Team,Away Team,Goals Home,Goals Away,Odd Home,Odd Draw,Odd Away,Best Tip,Best Tip Trust,Best Tip Odd,Underover,Trust Underover,Odd Underover,Final Result,Trust Final Result,tipOdd
Date,,,,,,,,,,,,,,,,,,,
2022-01-16,00:00,Liga Nacional,Guatemala,Antigua GFC,Achuapa,1,0,1.30,4.35,7.00,1,83,1.30,under 2.5,33.8463,1.80,1,83.3333,1.30
2022-01-16,01:00,Liga MX,Mexico,Tigres UANL,Puebla,0,2,1.58,3.80,5.50,1,84,1.58,under 2.5,13.0000,1.73,1,84.7810,1.58
2022-01-16,03:00,Liga MX,Mexico,Cruz Azul,FC Juarez,1,0,1.58,3.80,5.50,over 1.5,66,1.39,over 1.5,66.2010,1.39,1,40.7193,1.58
2022-01-16,03:06,Liga MX,Mexico,Club Tijuana,Leon,1,1,2.65,3.20,2.57,X2,51,1.41,under 2.5,22.3168,1.69,2,39.3339,2.57
2022-01-16,10:00,TFF 2. Lig,Turkey,Afjet Afyonspor,BB Bodrumspor,0,2,1.65,3.75,3.90,1,56,1.65,over 2.5,53.0470,1.75,1,56.8148,1.65


In [6]:

df.shape


(93492, 19)

In [7]:

print(df.columns)


Index(['Time', 'League', 'Country', 'Home Team', 'Away Team', 'Goals Home',
       'Goals Away', 'Odd Home', 'Odd Draw', 'Odd Away', 'Best Tip',
       'Best Tip Trust', 'Best Tip Odd', 'Underover', 'Trust Underover',
       'Odd Underover', 'Final Result', 'Trust Final Result', 'tipOdd'],
      dtype='object')


In [8]:
df.dtypes

Time                   object
League                 object
Country                object
Home Team              object
Away Team              object
Goals Home              int64
Goals Away              int64
Odd Home              float64
Odd Draw              float64
Odd Away              float64
Best Tip               object
Best Tip Trust          int64
Best Tip Odd          float64
Underover              object
Trust Underover       float64
Odd Underover         float64
Final Result           object
Trust Final Result    float64
tipOdd                float64
dtype: object

In [9]:

# Handle missing values if any
df.dropna(inplace=True)  # For demonstration, dropping rows with missing values

# Encode categorical variables
df= pd.get_dummies(df, columns=["League", "Country", "Home Team", "Away Team"])


In [10]:
# Normalize numerical variables
scaler = StandardScaler()
numerical_cols = ["Goals Home", "Goals Away", "Odd Home", "Odd Draw", "Odd Away", "Best Tip Odd", "Odd Underover"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


In [11]:
# Feature Engineering
df['goal_difference'] = df['Goals Home'] - df['Goals Away']
df['odd_difference'] = df['Odd Home'] - df['Odd Away']

In [12]:

# Drop irrelevant columns
df.drop(['Time'], axis=1, inplace=True)

In [13]:
# Save preprocessed dataset
df.to_csv("preprocessed_dataset.csv", index=False)


In [14]:

# Downcast numeric data types
numeric_cols = df.select_dtypes(include=['int', 'float']).columns
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, downcast='integer')  # or 'float'

# Encode categorical variables with sparse matrix
categorical_cols = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categorical_cols, sparse=True)

In [15]:
# Store the compressed dataset
df.to_csv("compressed_dataset.csv.gz", compression="gzip", index=False)

In [16]:
# Load the compressed dataset
df = pd.read_csv("compressed_dataset.csv.gz")

In [17]:
# Define the features (X) and target variable (y)
X = df.drop(["Final Result_1", "Final Result_X", "Final Result_2"], axis=1)
y = df[["Final Result_1", "Final Result_X", "Final Result_2"]]  # Using one-hot encoded columns as target


In [18]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)


In [20]:
# Train the model
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [21]:
# Predict on the test set
y_pred = rf_classifier.predict(X_test)


In [22]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9291405957537836


In [23]:
from sklearn.metrics import classification_report, confusion_matrix

In [24]:
# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))



Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      8086
           1       0.99      0.32      0.48       801
           2       0.99      0.90      0.94      4368

   micro avg       0.99      0.90      0.94     13255
   macro avg       0.99      0.72      0.80     13255
weighted avg       0.99      0.90      0.94     13255
 samples avg       0.64      0.64      0.64     13255



c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1)))

Confusion Matrix:
[[13492     1    37]
 [  532   253    16]
 [  437     1  3930]]


In [27]:
import joblib

# Define the file path to save the model
model_filename = 'football_match_prediction_model.pkl'

# Save the model to disk
joblib.dump(rf_classifier, model_filename)

print("Model saved successfully as", model_filename)


Model saved successfully as football_match_prediction_model.pkl
